In [6]:
%load_ext autoreload
%autoreload 1
%aimport numa_teaching.formatting
from numa_teaching.formatting import fltx, nan_sp

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Otra forma de integración numérica

In [7]:
# sympy
from sympy import init_printing
from sympy import I, nroots, lambdify, diff, latex, symbols, pi, interpolating_poly, cos, N, integrate, Integral, exp, solve
from sympy.abc import x, epsilon, alpha
import operator
import functools
init_printing()
#init_printing(use_latex=False)

In [8]:
# numpy, pandas, matplotlib
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

%matplotlib inline
plt.rcParams["figure.figsize"] = [12, 9] # ancho, alto de figuras
plt.rcParams["font.size"] = 20

In [9]:
# Jupyter/ IPython formatting
from IPython.display import Math, Latex

In [10]:
from collections import namedtuple
from itertools import zip_longest
from functools import partial

Latex macros
$$ 
\newcommand{\d}{\mathrm{d}}
\DeclareMathOperator{\deg}{deg}
$$

## Otra forma de constuir intagración numérica

Buscamos expresiones
$$
\int_{-1}^1 f(x)\,\d x \approx \sum_{j=1}^n w_j f(x_j)
$$
que sean **exactas** para polinomios del grado más alto posible. No imponemos restricciones a los nodos $\{ x_j \}$ o pesos $\{ w_j \}$. Esperamos que la formula se muy precisa para funciones que se pueden aproximar bien por polinomios.

No hay garantia que funciona. En general es mala idea trabajar con nodos $\{ x_j \}$ equidistantes.

**Caso $n = 1$**: Tenemos
$$
w_1 f(x_1) \approx \int_{-1}^1 f(x)\,\d x
$$
Ahora hay que determinar $w_1$ y $x_1$. Son 2 variables, por lo tanto imponemos una solución exacta para $f(x) = 1$ y para $f(x) = x$:
\begin{align}
w_1 \cdot 1 &= \int_{-1}^1 1\,\d x\\
w_1 &= 2
\end{align}
De $f(x) = x$ obtenemos
\begin{align}
w_1 \cdot x_1 &= \int_{-1}^1 x\,\d x\\
x_1 &= 0
\end{align}
La regla deseada es
$$
\int_{-1}^1 f(x)\,\d x \approx 2f(0)
$$
es la *regla del punto en medio*

**Caso $n = 2$**: La formula es
$$
w_1 f(x_1) + w_2 f(x_2) \approx \int_{-1}^1 f(x)\,\d x
$$
Tenemos que determinar los pesos $w_1$, $w_2$ y los nodos $x_1$, $x_2$. Para eso usamos
$$
f(x) = 1,\, x,\, x^2,\, x^3
$$
que nos da el sistema de ecuaciones
\begin{align}
w_1 + w_2 &= \int_{-1}^1 1\,\d x = 2\\
w_1 x_1 + w_2 x_2 &= \int_{-1}^1 x\,\d x = 0\\
w_1 x_1^2 + w_2 x_2^2 &= \int_{-1}^1 x^2\,\d x = \frac{2}{3}\\
w_1 x_1^3 + w_2 x_2^3 &= \int_{-1}^1 x^3\,\d x = 0
\end{align}


La solución es
$$
w_1 = w_2 = 1, \qquad x_1 = \frac{-1}{\sqrt 3}, \qquad x_2 = \frac{1}{\sqrt 3}
$$
que nos genera la formula
$$
\int_{-1}^1 f(x)\,\d x \approx f\Bigl( \frac{-1}{\sqrt 3} \bigr) + f\Bigl( \frac{1}{\sqrt 3} \bigr)
$$
Se dice que tiene *grado de precisión 3* porque integra exactamente todos los polinomios de grado $\leq 3$. No integra exactamente a $f(x) = x^4$:
$$
\int_{-1}^1 x^4\,\d x = \frac{2}{5}
$$
$$
f\Bigl( \frac{-1}{\sqrt 3} \bigr) + f\Bigl( \frac{1}{\sqrt 3} \bigr) = \frac{2}{9}
$$

### Ejemplo

Integramos
$$
\int_{-1}^1 \frac{\d x}{3 + x} \log 2 \doteq 0.69314718
$$
Comparado con
$$
\frac{1}{3 + x_1} + \frac{1}{3 + x_2} = 0.69230769
$$
$$
\text{Error} = 0.000839
$$

## Caso general

Tenemos que encontrar los pesos $\{ w_j \}$ y los nodos  $\{ x_j \}$ para
$$
\int_{-1}^1 f(x)\,\d x \approx \sum_{j=1}^n w_j f(x_j)
$$
sea exacto para polinomios del grado más alto posible. Tenemos $2n$ desconocidos, así que hay que imponer $2n$ condiciones. Pedimos que la formula sea exacta para
$$
f(x) = x^i,\qquad i = 0, 1, 2, \dots, 2n - 1
$$
Nos genera el sistema de ecuaciones
$$
w_1 x_1^i + w_2 x_2^i + \dots + w_n x_n^i = \int_{-1}^1 x^i\,\d x
$$
que tiene te mano derecha
$$
\int_{-1}^1 x^i\,\d x = 
  \begin{cases}
    \frac{2}{i + 1}, & i = 0, 2, \dots, 2n - 2\\
    0,               & i = 1, 3, \dots, 2n - 1
  \end{cases}
$$

El sistema de ecuaciones
$$
w_1 x_1^i + w_2 x_2^i + \dots + w_n x_n^i = \int_{-1}^1 x^i\,\d x
$$
tiene una solución única, modulo ordenamiento de los indices. La regla de integración que genera es la *quadratura de Gauss*.

En la practica, uno encuentra los pesos y nodos de otra manera en vez de resolver este sistema de ecuaciones. Nodos y pesos tienen otros propriedades que se utiliza. La mayoría de las bibliotecas númericas cuenten con subroutinas para generarlas.

In [11]:
from numpy.polynomial.legendre import leggauss
leggauss(3) # Nodos y pesos

(array([-0.77459667,  0.        ,  0.77459667]),
 array([0.55555556, 0.88888889, 0.55555556]))

In [12]:
I = sum([w * 1/(3 + x) for x, w in zip(*leggauss(3))]) 
I, I - np.log(2)

In [13]:
from scipy.integrate import quadrature, fixed_quad
ff = lambda x: 1/(3 + x)
fixed_quad(ff, -1, 1, n=3)

(0.693121693121693, None)

In [14]:
quadrature(ff, -1, 1, tol=1e-5, rtol=1e-5) # Adaptive

## Simetría

Nodos y pesos tienen simetrías, en particular
$$
x_i = -x_{n-i},\qquad w_i = w_{n-i}, \qquad i = 1, 2, \dots, n
$$

Es posible demonstrar que todos los pesos cumplen
$$
w_i > 0
$$
para todos los $i > 0$. Es una propriedad deseable del punto de vista práctico y permita desarollar expresiones útiles par alos errores.

## Cambio del intervalo de integración

La integración sobre el intervalo $[a, b]$ se relaciona con la integración sobre $[-1, 1]$ utilizando
$$
\int_a^b F(x)\,\d x = \frac{b - a}{2} \int_{-1}^1 F\left( \frac{b + a + t (b - a)}{2} \right)\,\d t
$$
basado en el cambio de variables
$$
x = \frac{b + a + t (b - a)}{2}, \qquad -1 \leq t \leq 1
$$

### Ejemplo

En el intervalo $[0, \pi]$ usamos
$$
x = (1 + t) \frac{\pi}{2}
$$
$$
\int_0^\pi F(x)\,\d x = \frac{\pi}{2} \int_{-1}^1 F\bigl( (1 + t)\frac{\pi}{2} \bigr)\,\d t
$$

In [15]:
def I1(x):
    return np.exp(-x**2)
I1x = 0.74682413281234

def I2(x):
    return 1 / (1 + x**2)
I2x = np.arctan(4)

def I3(x):
    return 1/(2 + np.cos(x))
I3x = 2*np.pi / np.sqrt(3)

tabla = []
for i in list(range(2, 8)) + [10, 15, 20]:
    tabla.append([i, 
                  fixed_quad(I1, 0, 1, n=i)[0] - I1x,
                  fixed_quad(I2, 0, 4, n=i)[0] - I2x,
                  fixed_quad(I3, 0, 2*np.pi, n=i)[0] - I3x,
                 ])

pd.DataFrame(tabla).style.format("{:.6g}")

,0,1,2,3
0,2,-0.000229445,0.0232948,-0.82338
1,3,-9.54862e-06,0.0348995,0.429851
2,4,3.35319e-07,0.00189556,-0.176613
3,5,-6.04609e-09,-0.00170485,0.0812424
4,6,7.78155e-11,-0.000273601,-0.0354608
5,7,-7.01661e-13,6.45093e-05,0.0158287
6,10,8.71525e-14,-1.27061e-06,-0.00137198
7,15,8.69305e-14,-7.4042e-10,2.33153e-05
8,20,8.70415e-14,-3.16414e-13,-3.95577e-07


## Una formula para el error de integración Gaussiana

El error para la quadratura gaussiana está dado por
$$
E_n(f) = \int_{-1}^1 f(x)\,\d x - \sum_{j=1}^n w_j f(x_j)
$$
con
\begin{align}
E_n(f) &= e_n \frac{f^{(2n)}(c_n)}{(2n)!}\\
e_n    &= \frac{2^{2n + 1} (n!)^4}{(2n + 1) [(2n)!]^2} \approx \frac{\pi}{4^n}
\end{align}
para algún $c_n \in [a, b]$. Esta forma no es muy intuitiva. Para entenderla mejor introducimos
$$
M_k = \max_{-1 \leq x \leq 1} \frac{\bigl| f^{(k)}(x) \bigr|}{k!}
$$


Con muchas funciones $f(x)$, la secuencia $\{ M_k \}$ es acotada o converge a cero. Por ejemplo
$$
f(x) = 
  \begin{cases}
  \cos x\\
  \frac{1}{2 + x}
  \end{cases}
\Rightarrow
M_k \leq
  \begin{cases}
  \frac{1}{k!}\\
  1
  \end{cases}
$$
Nos da para la formula de errores
\begin{align}
E_n(f)   &= e_n \frac{f^{(2n)}(c_n)}{(2n)!}\\
|E_n(f)| &\leq e_n M_{2n}
\end{align}
Usando
$$
e_n \approx \frac{\pi}{4^n}
$$
y el hecho que $\{ M_k \}$ es acotado para ver que el error baja un factor $4$ con cada incremento $ n \to n + 1$. Nos explica la convergencia rápida en comparación  con los metdos del trapezoide o de Simpson.

##  Una segunda formula para el error

Para $f(x)$ continua en $[a, b]$ y $n \geq 1$ tenemos para la quadratura de Gauss
$$
I \equiv \int_{-1}^1 f(x)\,\d x \approx \sum_{j=1}^n w_j f(x_j) \equiv I_n
$$
la relación
$$
\bigl| I(f) - I_n(f) \bigr| \leq 2 (b - a) \rho_{2n - 1}(f)
$$
donde $\rho_{2n - 1}(f)$ es el *error minimax* de grado $2n - 1$ para $f(x)$ en $[a, b]$:
$$
\rho_m(f) = \min_{\deg(p) \leq m} \left[ \max_{a \leq x \leq b} |f(x) - p(x)| \right], \qquad m \geq 0
$$

### Ejemplo

Consideramos $f(x) = e^{-x^2}$. Los errores minimax $\rho_m(f)$ son

| $m$   | $\rho_m(f)$ | $m$   | $\rho_m(f)$ |
| -----:| -----------:| -----:| -----------:|
| 1     | 5.30E − 2   | 6     | 7.82E − 6   |
| 2     | 1.79E − 2   | 7     | 4.62E − 7   |
| 3     | 6.63E − 4   | 8     | 9.64E − 8   |
| 4     | 4.63E − 4   | 9     | 8.05E − 9   |
| 5     | 1.62E − 5   | 10    | 9.16E − 10  |

Con esta tabla y la formula anterior podemos estimar el error para la integral
$$
I = \int_0^1 e^{-x^2}\,\d x
$$
para $n = 3$:
$$
|I - I_3| \leq 2\rho_5\Bigl( e^{-x^2} \Bigr) \doteq 3.24 \times 10^{-5}
$$
El error correcto es $9.55 \times 10^{-6}$.

## Integración de funciones no suaves

Evaluamos la integral 
$
I = \int_0^1 \sqrt{x}\,\d x = \frac{2}{3}
$
con cuadratura de Gauss.

In [16]:
# tabla
tab_sqrt = []
err_prev = np.nan
for n in 2**np.arange(1, 7):
    I_n = fixed_quad(np.sqrt, 0, 1, n=n)[0]
    err = 2./3. - I_n 
    ratio = err_prev / err
    tab_sqrt.append([n, err, ratio])
    err_prev = err

tab_sqrt = pd.DataFrame(tab_sqrt, columns=["$n$", "$I - I_n$", "Ratio"])

In [17]:
tab_sqrt.style.format(dict(zip(tab_sqrt.columns, ["{:g}", "{:.2e}", partial(nan_sp, fmt="{:.1f}")])))

,$n$,$I - I_n$,Ratio
0,2,-7.22e-03,
1,4,-1.16e-03,6.2
2,8,-1.69e-04,6.9
3,16,-2.30e-05,7.4
4,32,-3.00e-06,7.7
5,64,-3.84e-07,7.8


La última columna da 
$$
(I - I_{\frac{1}{2}n}) / (I - I_n) \approx 8 = 2^3
$$
que es consistente con $I - I_n \approx \frac{c}{n^3}$, que se puede demonstrar formalmente. En comparación, para las reglas del trapezoide y de Simpson, $I - I_n \approx \frac{c}{n^{1.5}}$

## Cuadratura de Gauss con peso

Consideramos integrales como
$$
\int_0^1 f(x) \log x\,\d x,\qquad
\int_0^1 x^\frac{1}{3} f(x)\,\d x
$$
Podemos considerar formulas de integración 
$$
\int_a^b w(x) f(x)\,\d x \approx \sum_{j=1}^n w_j f(x_j)
$$
donde es $f(x)$ una función con varias derivadas continuas. La función $w(x)$ puede tener singularidades, pero tiene que ser integrable. El intervalo de integración $[a, b]$ es finito. La función $w(x)$ es la *función de peso* y se absorbe en las definiciones de los pesos $\{ w_i \}$. Determinamos los nodos $\{ x_i \}$ y los pesos $\{ w_i \}$ para hacer que la formula sea exacta cuando $f(x)$ es un polinomio del grado más alto posible.

La formula para integración númerica
$$
\int_a^b w(x) f(x)\,\d x \approx \sum_{j=1}^n w_j f(x_j)
$$
se llama *cuadratura de Gauss con función de peso $w(x)$*. Para determinar los nodos $\{ x_i \}$ y los pesos $\{ w_i \}$ pedimos que sea exacta para
$$
f(x) = x^i, \qquad i = 0, 1, 2, \dots, 2n - 1
$$
Para ilustrar consideramos el caso
$$
\int_0^1 x^\frac{1}{3} f(x)\,\d x \approx \sum_{j=1}^n w_j f(x_j)
$$


**Caso $n = 1$**: Tenemos
$$
w_1 f(x_1) \approx \int_0^1 x^\frac{1}{3} f(x)\,\d x
$$
Ahora hay que determinar $w_1$ y $x_1$. Son 2 variables, por lo tanto imponemos una solución exacta para $f(x) = 1$ y para $f(x) = x$:
$$
w_1 = \int_0^1 x^\frac{1}{3}\,\d x = \frac{3}{4}
$$
De $f(x) = x$ obtenemos
\begin{align}
w_1 x_1 &= \int_0^1 x^\frac{1}{3} x\,\d x = \frac{3}{7}\\
x_1 &= \frac{4}{7}
\end{align}
La regla deseada es
$$
\int_0^1 x^\frac{1}{3} f(x)\,\d x \approx \frac{3}{4} f\bigl( \frac{4}{7} \bigr)
$$
con un grado de precisión 1.

**Caso $n = 2$**: La formula es
$$
w_1 f(x_1) + w_2 f(x_2) \approx \int_0^1 x^\frac{1}{3} f(x)\,\d x
$$
Tenemos que determinar los pesos $w_1$, $w_2$ y los nodos $x_1$, $x_2$. Para eso usamos
$$
f(x) = 1,\, x,\, x^2,\, x^3
$$
que nos da el sistema de ecuaciones
\begin{align}
w_1 + w_2 &= \int_0^1 x^\frac{1}{3} \,\d x = 2\frac{3}{4}\\
w_1 x_1 + w_2 x_2 &= \int_0^1 x^\frac{1}{3} x\,\d x = \frac{3}{7}\\
w_1 x_1^2 + w_2 x_2^2 &= \int_0^1 x^\frac{1}{3} x^2\,\d x = \frac{3}{10}\\
w_1 x_1^3 + w_2 x_2^3 &= \int_0^1 x^\frac{1}{3} x^3\,\d x = \frac{3}{13}
\end{align}


La solución es
\begin{gather}
x_1 = \frac{7}{13} - \frac{3}{65} \sqrt{35}, \qquad  x_2 = \frac{7}{13} + \frac{3}{65} \sqrt{35}\\
w_1 = \frac{3}{8} - \frac{3}{393} \sqrt{35}, \qquad  w_2 = \frac{3}{8} + \frac{3}{393} \sqrt{35}
\end{gather}
que es numéricamente
\begin{gather}
x_1 = 0.2654117024, \qquad x2 = 0.8115113746\\
w_1 = 0.3297238792, \qquad w2 = 0.4202761208
\end{gather}

La formula
$$
\int_0^1 x^\frac{1}{3} f(x)\,\d x \approx w_1 f(x_1) + w_2 f(x_2)
$$
tiene *grado de precisión 3*.

### Ejemplo

Integramos
$$
\int_0^1 x^\frac{1}{3} \cos x \,\d x  \doteq 0.6076257393
$$
Con la expresión anterior obtenemos
$$
w_1 f(x_1) + w_2 f(x_2) = 0.6074977951
$$
que tiene un error de $E_2 = 0.000128$.
Es un resultado bastane bueno, dado el esfuerzo computacional limitado. En comparación, los metodos trapezoidales y de Simpson convergen lentamente debido a la singularidad en la primera derivada en el origin.

### Generadores de pesos y nodos en SciPy

Roots and weights for orthogonal polynomials (scipy.special) *Note* Name changes in recent version of scipy.

| Name | Description | Funcion de peso $w(x)$ |
|:--- |:--- |:--- |
| c_roots(n[, mu]) | Gauss-Chebyshev (first kind) quadrature. | $1 / \sqrt{1 - (x/2)^2}$ |
| cg_roots(n, alpha[, mu]) | Gauss-Gegenbauer quadrature. | $ (1 - x^2)^{\alpha - 1/2} $|
| h_roots(n[, mu]) | Gauss-Hermite (physicst’s) quadrature. | $e^{-x^2}$ |
| he_roots(n[, mu]) | Gauss-Hermite (statistician’s) quadrature. | $e^{-(x/2)^2}$  |
| j_roots(n, alpha, beta[, mu]) | Gauss-Jacobi quadrature. | $(1 - x)^\alpha (1 + x)^\beta$ |
| js_roots(n, p1, q1[, mu]) | Gauss-Jacobi (shifted) quadrature. | $( 1− x)^{p − q} x^{q − 1} $|




| Name | Description | Funcion de peso $w(x)$ |
|:--- |:--- |:--- |
| l_roots(n[, mu]) | Gauss-Laguerre quadrature. | $e^{-x}$ |
| la_roots(n, alpha[, mu]) | Gauss-generalized Laguerre quadrature. | $x^\alpha e^{-x}$ |
| p_roots(n[, mu]) | Gauss-Legendre quadrature. | 1 |
| ps_roots(n[, mu]) | Gauss-Legendre (shifted) quadrature. | 1 |
| s_roots(n[, mu]) | Gauss-Chebyshev (second kind) quadrature. | $\sqrt{1 - (x/2)^2}$ |
| t_roots(n[, mu]) | Gauss-Chebyshev (first kind) quadrature. | $1/\sqrt{1 - x^2}$ |
| ts_roots(n[, mu]) | Gauss-Chebyshev (first kind, shifted) quadrature. | $1/\sqrt{x - x^2}$ |
| u_roots(n[, mu])| Gauss-Chebyshev (second kind) quadrature. | $\sqrt{1 - x^2}$ |
| us_roots(n[, mu]) | Gauss-Chebyshev (second kind, shifted) quadrature. |  $ \sqrt{x - x^2} $|


## Cambio de variables

En el ejemplo anterior podemos hacer el cambio de variable $x = t^3$ para transformar la integral a
$$
3 \int_0^1 t \cos (t^3) \,\d t
$$
Las reglas del trapezoide y de Simpson convergen más rapidamente, pero no tan rápido como el metodo de Gauss con peso.

En general, es posible mejorar el rendimiento de un metodo de integración con un cambio de variables, en general para mejorar la diferenciabilidad del integrante.

**Ejemplo** Usamos $x = t^r$, $r >1$:

$$
\int_0^1 g(x) \log x \,\d x = r\int_0^1 t^{r - 1} g(t^r) \log t \,\d t
$$